<a href="https://colab.research.google.com/github/RohanSaxena14/Sentiment-Analysis/blob/master/Sentiment_analysis_using_Tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **We will be using "IMDB reviews dataset" for training our model.**


1.   **Here we are first loading the dataset using pandas**
2.   **Then we will clean all reviews as they have HTML tags in them.**



In [0]:
import pandas as pd
from bs4 import BeautifulSoup

df = pd.read_csv("IMDB Dataset.csv")


for i in range(50000):
  html = df.iloc[i, 0]
  soup = BeautifulSoup(html)
  text = soup.get_text()
  df.iloc[i, 0] = text

**This is how our data looks like.**

In [0]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#**We will be converting sentences into vectors using TfidfVectorizer**
**to learn more about tf-idf, I refer this blog post here.**

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XpVol8gzbzB

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 5000, max_df = 0.7)
 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(list(df['review']))
tfidf_vectorizer_vectors = tfidf_vectorizer_vectors.todense()

#**After forming vectors, we now have to prepare numpy arrays which can be used for training and testing purposes**

In [0]:
import numpy as np

labels = list(df["sentiment"])

labels = np.array([x=="positive" for x in labels]).astype(int)


train_vectors = np.array(tfidf_vectorizer_vectors[:25000])
test_vectors = np.array(tfidf_vectorizer_vectors[25000:])

train_labels = labels[:25000].reshape(25000, 1)
test_labels = labels[25000:]

**Here are the dimensions of our trainig data**

In [0]:
print(train_labels.shape)
print(train_vectors.shape)

(25000, 1)
(25000, 5000)


#**We will be using keras for building our neural network**

read more about it from here

https://victorzhou.com/blog/keras-neural-network-tutorial/

In [0]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(800, activation = 'relu', input_dim = train_vectors.shape[1]),
                             #tf.keras.layers.Dense(800, activation = 'relu'),
                             #tf.keras.layers.Dense(800, activation = 'relu'),
                             tf.keras.layers.Dense(200, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
model.fit(train_vectors, train_labels, epochs = 2, batch_size = 32, validation_split = 0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/2
22500/22500 [==============================] - 14s 625us/sample - loss: 0.3202 - accuracy: 0.8618 - val_loss: 0.2884 - val_accuracy: 0.8728
Epoch 2/2
22500/22500 [==============================] - 14s 614us/sample - loss: 0.1859 - accuracy: 0.9252 - val_loss: 0.3549 - val_accuracy: 0.8660


#**After training for just 2 epochs, we are able to achieve 87% accuracy on test data.**

In [0]:
loss, accuracy = model.evaluate(test_vectors, test_labels)
print("Loss :- ", loss)
print("Accuracy :- ", accuracy)


# Accuracy:- 87%

#**Lastly we would like to see, how well a simple Logistic Regressor can fit our data and it turns out that it is achieving slightly greater Accuracy over Neural network model.**

**Note:- It is only because we are using a shallow neural net and its hyper parameters are not tuned very well, so feel free to experiment with it and you will deffinately be able to achieve much higher accuracy.**

In [0]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_vectors, train_labels)
score = classifier.score(test_vectors, test_labels)

print("Accuracy:", score)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.88824
